In [26]:
# Download and Clean Dataset

import pandas as pd
import numpy as np

concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [27]:
# Let's check how many data points we have
concrete_data.shape

(1030, 9)

In [28]:
#Let's check the dataset for any missing values
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [29]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [30]:
# Let's split data into predictors and target

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [31]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [32]:
# Let's save the number of predictors to n_cols since we will need this number when building our network.

n_cols = predictors.shape[1] # number of predictors
print(n_cols)

8


In [33]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [34]:

# Let's ormalize the data by substracting the mean and dividing by the standard deviation.

predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()



,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



#Let's creates arrays of independent (X) and dependent (y) variables, respectively. 
X = predictors_norm.values
y = target.values


In [38]:
# Let's build a Neural Network in Keras using regression model function

from keras.models import Sequential
from keras.layers import Dense

def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])
    return model


In [39]:
# Define the MSE list to keep calculated mean_squared_errors between the predicted concrete strength and the actual concrete strength
# increase number of epochs from 50 to 100

MSE = []
# Let's collect 50 mean squared errors for each randomly splitted train and test data sets 
for x in range(50):
    
    print("itteration : ", x)
    
    #  Randomly split the data into a training and test sets by holding 30% of the data for testing.

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    # Create and fit model using 20% of training data for validation
    model = regression_model()
    history = model.fit(X_train, y_train, epochs=50,validation_split=0.2, verbose = 0)

    #  Predict on the Test Data and Compute mean_squared_error between the predicted concrete strength and the actual concrete strength
    pred_test= model.predict(X_test)
    MSE.append(mean_squared_error(y_test,pred_test))
    

itteration :  0
itteration :  1
itteration :  2
itteration :  3
itteration :  4
itteration :  5
itteration :  6
itteration :  7
itteration :  8
itteration :  9
itteration :  10
itteration :  11
itteration :  12
itteration :  13
itteration :  14
itteration :  15
itteration :  16
itteration :  17
itteration :  18
itteration :  19
itteration :  20
itteration :  21
itteration :  22
itteration :  23
itteration :  24
itteration :  25
itteration :  26
itteration :  27
itteration :  28
itteration :  29
itteration :  30
itteration :  31
itteration :  32
itteration :  33
itteration :  34
itteration :  35
itteration :  36
itteration :  37
itteration :  38
itteration :  39
itteration :  40
itteration :  41
itteration :  42
itteration :  43
itteration :  44
itteration :  45
itteration :  46
itteration :  47
itteration :  48
itteration :  49


In [40]:
# Convert resulted list to numpy array
MSE_arr = np.array(MSE)

MSE_mean = np.mean(MSE_arr)
# ddof=1 interprete it as samples, i.e. estimate true variance
MSE_std = np.std(MSE_arr,ddof=1)
print("mean_squared_error = ",round(MSE_mean,2)," standard deviation = ", round(MSE_std,2))

## Task A - mean_squared_error =  166.97  standard deviation =  119.04


mean_squared_error =  167.13  standard deviation =  15.74


In [ ]:
# With normalized version of the data - mean of mean_squared_errors between the predicted concrete strength and the actual concrete strength
# didn't change, but  standard deviation decreased by ~7x